# Model trained 27/06

Predicted   0.0   1.0   All</br>
True                       </br>
0.0        2094   110  2204</br>
1.0         463  2599  3062</br>
All        2557  2709  5266</br>

test accuracy for epoch 19 is: 0.8911887407302856
test Precision for epoch 19 is: 0.959394633769989
test Recall for epoch 19 is: 0.8487916588783264
test F1 for epoch 19 is: 0.9007104635238647 

Validation accuracy for epoch 19 is: 0.8708697557449341
Validation Precision for epoch 19 is: 0.9584295749664307
Validation Recall for epoch 19 is: 0.8131939768791199
Validation F1 for epoch 19 is: 0.879858672618866 

Training accuracy for epoch 19 is: 0.8711806535720825
Training Precision for epoch 19 is: 0.962721049785614
Training Recall for epoch 19 is: 0.8130359649658203
Training F1 for epoch 19 is: 0.8815697431564331 

In [1]:
from datetime import datetime, timedelta
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from stocksent import Sentiment
import torch
import torch.utils.data as Data
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import yahoo_fin.stock_info as si

c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
snp500_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/snp500'))
print(len(snp500_files))
nasdaq_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/nasdaq')) - snp500_files
print(len(nasdaq_files))
nyse_files: set = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/nyse')) - snp500_files
print(len(nyse_files))
nasdaq_nyse: set = nasdaq_files.union(nyse_files)
print(len(nasdaq_nyse))


In [ ]:
initial_stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/nasdaq_snp500_nyse/snp500/AMZN.csv')
initial_stock_data['Date'] = initial_stock_data['Date'].apply(lambda x: x[-4:])
annual_data = initial_stock_data.groupby('Date').mean()
annual_data['Annual Percent Change'] = annual_data['Close'].pct_change()
annual_data = annual_data.fillna(0)
print(annual_data.head(20))

In [148]:
news_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/analyst_ratings_processed.csv', index_col=0)
news_data['date'] = news_data['date'].apply(lambda x: str(x)[:4])
print(news_data.head(30))

                                                  title  date stock
0.0             Stocks That Hit 52-Week Highs On Friday  2020     A
1.0          Stocks That Hit 52-Week Highs On Wednesday  2020     A
2.0                       71 Biggest Movers From Friday  2020     A
3.0        46 Stocks Moving In Friday's Mid-Day Session  2020     A
4.0   B of A Securities Maintains Neutral on Agilent...  2020     A
5.0   CFRA Maintains Hold on Agilent Technologies, L...  2020     A
6.0   UBS Maintains Neutral on Agilent Technologies,...  2020     A
7.0   Agilent Technologies shares are trading higher...  2020     A
8.0   Wells Fargo Maintains Overweight on Agilent Te...  2020     A
9.0          10 Biggest Price Target Changes For Friday  2020     A
10.0    30 Stocks Moving in Friday's Pre-Market Session  2020     A
11.0  SVB Leerink Maintains Outperform on Agilent Te...  2020     A
12.0  8 Stocks Moving In Thursday's After-Hours Session  2020     A
13.0  Agilent Technologies shares are trading hi

In [159]:
news_by_stock = news_data.groupby('stock')
already_processed = set(os.listdir('G:/My Drive/UdS/Classes/Data Science/Old/stock_data'))
for stock in tqdm(news_by_stock):
    # if stock[0] + '.csv' in already_processed:
    #     continue
    try:
        stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/snp500/' + stock[0] + '.csv')
        label = 1
    except FileNotFoundError:
        try:
            stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/nasdaq/' + stock[0] + '.csv')
            label = 0
        except FileNotFoundError:
            try:
                stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/Old/nasdaq_snp500_nyse/nyse/' + stock[0] + '.csv')
                label = 0
            except FileNotFoundError:
                continue
    stock_data['Date'] = stock_data['Date'].apply(lambda x: x[-4:])
    
    annual_data = stock_data.groupby('Date')
    placeholder = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
    for year in annual_data:
        try:
            year[1]['Annual Percent Change'] = (year[1].iloc[-1]['close'] - year[1].iloc[0]['close']) / year[1].iloc[0]['close'] * 100
        except:
            print(year[1])
        means = year[1].mean()
        means = means.fillna(0)
        placeholder = pd.concat([placeholder, means], ignore_index=True)


    news_by_year = stock[1].groupby('date')
    sentiments_frame = pd.DataFrame(columns=['positive', 'negative', 'neutral'])
    sentiment_index = [year[0] for year in news_by_year]
    for year in news_by_year:
        stories = [year[1]['title'].iloc[:5].tolist()][0]

        sentiments = pd.DataFrame(columns=['positive', 'negative', 'neutral'], index=[year[0]])
        for story in stories:
            sentiment = get_sentiment(story)
            sentiment_frame = pd.DataFrame({'positive': sentiment[0], 'negative': sentiment[1], 
                               'neutral': sentiment[2]}, index=[year[0]])
            sentiments = pd.concat([sentiments, sentiment_frame], ignore_index=True)
        
        mean_sentiments = sentiments.mean()
        sentiments_frame = pd.concat([sentiments_frame, mean_sentiments], ignore_index=True)
    sentiments_frame.index = sentiment_index
    combined_data = pd.concat([placeholder, sentiments_frame], axis=1)
    combined_data = combined_data.fillna(combined_data.mean())
    combined_data['Label'] = label
    combined_data = combined_data.reset_index(drop=True)
    combined_data.drop(combined_data[combined_data.Volume == 0].index, inplace=True)
    print(combined_data.head(20))
    # combined_data.drop(columns=['Date'], inplace=True)
    # combined_data.to_csv(f'G:/My Drive/UdS/Classes/Data Science/stock_data/{stock[0]}.csv', index=False)
    break

  0%|          | 0/6192 [00:00<?, ?it/s]

    Date        Low       Open    Volume       High      Close  Adjusted Close
0   1999  28.612303  32.546494  62546380  35.765381  31.473534       26.929760
1   1999  28.478184  30.713518  15234146  30.758226  28.880545       24.711119
2   1999  28.657009  29.551144   6577870  31.473534  31.473534       26.929760
3   1999  28.612303  30.400572   5975611  31.205294  28.612303       24.481602
4   1999  28.612303  28.701717   4843231  29.998213  29.372318       25.131901
5   1999  29.148785  29.238197   1729466  29.685265  29.461731       25.208408
6   1999  29.014664  29.327612   4074751  30.355865  30.132332       25.782187
7   1999  29.282904  30.042917   4310034  30.713518  30.177038       25.820436
8   1999  29.953505  30.177038   2957329  31.071173  30.713518       26.279463
9   1999  30.892345  31.294706   3069868  32.188843  31.562946       27.006264
10  1999  31.697067  32.144135   3041629  32.680614  31.831186       27.235783
11  1999  32.322960  32.367668   2250780  33.217094 

  0%|          | 0/6192 [00:03<?, ?it/s]


ValueError: Length mismatch: Expected axis has 36 elements, new values have 12 elements

### Relabel stocks by APR

In [141]:
#relabelled_stocks = os.listdir('G:/My Drive/UdS/Classes/Data Science/relabelled_data')
good_years = 0
for stock in tqdm(os.listdir('G:/My Drive/UdS/Classes/Data Science/Old/truncated_data')):
    if stock == "desktop.ini":
        continue
    # if stock in relabelled_stocks:
    #     continue
    try:
        truncated_data = pd.read_csv(f'G:/My Drive/UdS/Classes/Data Science/Old/truncated_data/{stock}')
    except:
        continue
    for year in truncated_data.index.to_list():
        apr = truncated_data.loc[year, 'Annual Percent Change']
        if apr > 3.8:
            truncated_data.loc[year, 'Label'] = 2
            good_years += 1
        elif apr > 0:
            truncated_data.loc[year, 'Label'] = 1
        else:
            truncated_data.loc[year, 'Label'] = 0
    truncated_data.to_csv(f'G:/My Drive/UdS/Classes/Data Science/relabelled_data/{stock}', index=False)

100%|██████████| 2337/2337 [01:54<00:00, 20.47it/s]


In [ ]:
prepared_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/stock_data')
#processed_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/truncated_data')

recent_year: dict = {}
for stock in tqdm(prepared_stock_list):
    if stock == 'desktop.ini':
        continue
    # if stock in processed_stock_list:
    #     continue
    stock_data = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/stock_data/' + stock)
    try:
        recent_year[stock[:-4]] = stock_data.iloc[-1]
    except IndexError:
        print(stock)
        print(stock_data)
        print(stock_data.head())
        continue
    # write to csv, truncating first and last line
    #stock_data[1:-1].to_csv('G:/My Drive/UdS/Classes/Data Science/truncated_data/' + stock, index=False)
    
recent_year_dataframe = pd.DataFrame.from_dict(recent_year, orient='index')

# recent_year_dataframe.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/toy_data/stonks.csv', index=True)

In [142]:
relabelled_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/relabelled_data')
#relabelled_stock_list = os.listdir('G:/My Drive/UdS/Classes/Data Science/truncated_data')

columns = ["Low","Open","Volume","High","Close","Adjusted Close","Annual Percent Change","positive","negative","neutral","Label"]
full_dataset = pd.DataFrame(columns=columns)
for stock in tqdm(relabelled_stock_list):
    if stock == 'desktop.ini':
        continue
    try:
        stock_data = pd.read_csv(f'G:/My Drive/UdS/Classes/Data Science/relabelled_data/{stock}')
    except:
        continue
    try:
        full_dataset = pd.concat([full_dataset, stock_data], ignore_index=True)
    except IndexError:
        print(stock)
        print(stock_data)
        print(stock_data.head())
        continue
# write to csv
full_dataset.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/stonks.csv', index=False)


100%|██████████| 2337/2337 [00:51<00:00, 44.97it/s]


In [3]:
stonks = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/stronks.csv')
groups = stonks.groupby('Label')
print(groups.size())

Label
0    21606
1    31046
dtype: int64


In [16]:
column_names: "list[str]" = ["low","open","volume","high","close","adjclose","Annual Percent Change","positive","negative","neutral","Label"]
stock_means: "list[float]" = [526.3256709784156, 543.3051050285828, 1550317.649111653, 560.6335547940919, 543.4596727527701, 432.9478474861315, 2.1972159402834595, 0.26918878308244654, 0.27109889272321963, 0.45971232487752245, 0.5896452176555497]
stock_std: "list[float]" = [25100.799536177463, 25956.76629687626, 13374912.758878224, 26792.649346820093, 25956.514595704837, 20572.229772641494, 477.38693948841035, 0.13956166294058042, 0.1719876441524078, 0.16237148015431316, 0.4919027653578207]

mean_series = pd.Series(stock_means, index=column_names)
std_series = pd.Series(stock_std, index=column_names)
print(mean_series)
print(std_series)



low                      5.263257e+02
open                     5.433051e+02
volume                   1.550318e+06
high                     5.606336e+02
close                    5.434597e+02
adjclose                 4.329478e+02
Annual Percent Change    2.197216e+00
positive                 2.691888e-01
negative                 2.710989e-01
neutral                  4.597123e-01
Label                    5.896452e-01
dtype: float64
low                      2.510080e+04
open                     2.595677e+04
volume                   1.337491e+07
high                     2.679265e+04
close                    2.595651e+04
adjclose                 2.057223e+04
Annual Percent Change    4.773869e+02
positive                 1.395617e-01
negative                 1.719876e-01
neutral                  1.623715e-01
Label                    4.919028e-01
dtype: float64


In [116]:
y = stonks["Label"]
X = stonks.drop(columns=["Label"])
X = (X-X.mean())/X.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80:20 train to test
X_test, X_dev, y_test, y_dev = train_test_split(X_test, y_test, test_size=0.5) # 50:50 test to dev > 80:10:10 train:dev:test

# write splits to individual csv files
X_train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_train.csv', index=False)
X_dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_dev.csv', index=False)
X_test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_test.csv', index=False)
y_train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_train.csv', index=False)
y_dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_dev.csv', index=False)
y_test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_test.csv', index=False)

### Reload data for training

In [117]:
# reload splits from csv files
X_train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_train.csv')
X_dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_dev.csv')
X_test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/X_test.csv')
y_train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_train.csv')
y_dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_dev.csv')
y_test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/relabelled_data/y_test.csv')

# verify data integrity
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)
print(y_train.shape)
print(y_dev.shape)
print(y_test.shape)


(42121, 10)
(5266, 10)
(5265, 10)
(42121, 1)
(5266, 1)
(5265, 1)


In [62]:
# train = pd.concat([X_train, y_train], axis=1)
# dev = pd.concat([X_dev, y_dev], axis=1)
# test = pd.concat([X_test, y_test], axis=1)

# write splits to individual csv files
# train.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/train.csv', index=False)
# dev.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/dev.csv', index=False)
# test.to_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/test.csv', index=False)

# reload splits from csv files
train = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/train.csv')
dev = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/dev.csv')
test = pd.read_csv('G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/finalized_data/test.csv')


In [118]:
print(train.shape)
print(dev.shape)
print(test.shape)

NameError: name 'train' is not defined

In [64]:
train_dataset = train.copy()
y_train_indices = train_dataset.index

y_train = [train_dataset.Label[i] for i in y_train_indices]
#y_train = train['Label']

class_sample_count = np.array(
    [len(np.where(y_train == t)[0]) for t in np.unique(y_train)])
print(class_sample_count)
print(y_train)
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)
sampler = Data.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_dataset.drop(columns=["Label"], inplace=True)
print(train_dataset.head())
train_dataset = Data.TensorDataset(torch.tensor(train_dataset.values), torch.tensor(y_train))
print(train_dataset[0])

[39783  6076]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [134]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train_X = torch.tensor(X_train.values, dtype=torch.float32)
# train_y = torch.tensor(y_train.values, dtype=torch.long)

class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

traindata = Data.TensorDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
print(traindata)
trainloader = Data.DataLoader(traindata, batch_size=64, shuffle=True)

train_dataloader = Data.DataLoader(traindata, batch_size=64, num_workers=0)


model = torch.nn.Sequential(
        torch.nn.Linear(10, 10),
        torch.nn.ReLU(),
        torch.nn.Linear(10, 1),
        torch.nn.Sigmoid(),
        )

model.train()

#loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = torch.nn.HingeEmbeddingLoss()
num_epochs = 100
prev_loss = 100000000.0
current_loss = 0.0

for n in range(num_epochs):
    epoch_loss = 0.0
    train_predictions = pd.DataFrame(columns=['y_pred', 'y_true'])
    for (batch_idx, batch) in enumerate(train_dataloader):
        X = batch[0].to(torch.float32)  # inputs
        Y = batch[1].to(torch.float32)  # correct class/label/politics
        y_vals = [val for item in Y.detach().numpy() for val in item]
        optimizer.zero_grad()
        oupt = model(X).squeeze(1).to(torch.float32)  # 1D tensor for each sample
        train_y_frame = pd.DataFrame({'y_pred': oupt.detach().numpy(), 'y_true': y_vals})
        train_predictions = pd.concat([train_predictions, train_y_frame], ignore_index=True)
        loss_val = loss_fn(oupt, torch.tensor(y_vals))  # a tensor
        epoch_loss += loss_val.item()  # accumulate
        loss_val.backward()
        optimizer.step()
    current_loss = epoch_loss
    confusion = pd.crosstab(train_predictions['y_true'], train_predictions['y_pred'], rownames=['True'], 
                        colnames=['Predicted'], margins=True)
    print(confusion)
    if current_loss >= prev_loss:
        print("Loss did not decrease, stopping training")
        break
    prev_loss = current_loss
    print("epoch = %d, loss = %f" % (n, epoch_loss))

# for n in tqdm(range(num_epochs)):
#     for X, y in train_dataloader:
#         X = X.to(torch.float32)
#         y = y
#         y_pred = model(X).squeeze(1)
#         loss = loss_fn(y_pred, y).item()
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

print(model)

torch.save(model, "G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/models/multilayer_model2.pickle")

KeyboardInterrupt: 

In [117]:
# dev_X = torch.tensor(X_dev.values, dtype=torch.float32)
# dev_y = torch.tensor(y_dev.values, dtype=torch.long)

dev_data = dev.copy()

y_dev = [dev_data.Label[i] for i in dev_data.index]
dev_data.drop(columns=["Label"], inplace=True)
dev_dataset = Data.TensorDataset(torch.tensor(dev_data.values), torch.tensor(y_dev))

class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        super(MyDataset, self).__init__()
        assert x.shape[0] == y.shape[0] # assuming shape[0] = dataset size
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]


dev_loader = Data.DataLoader(dev_dataset, batch_size=64)

model = torch.load("G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/models/multilayer_model.pickle")
print(model)
model.eval()

dev_predictions = pd.DataFrame(columns=['y_pred', 'y_true'])
with torch.no_grad():
    for X, y in dev_loader:
        X = X.to(torch.float32)
        y = y
        y_pred = model(X).squeeze(1)
        y_frame = pd.DataFrame({'y_pred': y_pred, 'y_true': y})
        dev_predictions = dev_predictions.append(y_frame, ignore_index=True)

confusion = pd.crosstab(dev_predictions['y_true'], dev_predictions['y_pred'], rownames=['True'], 
                        colnames=['Predicted'], margins=True)
print(confusion)

try:
    accuracy = (confusion[0][0] + confusion[1][1]) / confusion['All']['All']
except KeyError:
    accuracy = (confusion[0][0]) / confusion['All']['All']
print("accuracy:", accuracy)
try:
    precision = confusion[1][1] / confusion['All'][1]
except KeyError:
    precision = 0
print("precision:", precision)
try:
    recall = confusion[1][1] / confusion[1]['All']
except KeyError:
    recall = 0
print("recall:", recall)
try:
    f1 = 2 * (precision * recall) / (precision + recall)
except ZeroDivisionError:
    f1 = 0
print("f1:", f1)

Sequential(
  (0): Linear(in_features=10, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
  (5): LogSoftmax(dim=1)
)
Predicted   0.0   All
True                 
0          4970  4970
1           763   763
All        5733  5733
accuracy: 0.8669108669108669
precision: 0
recall: 0
f1: 0


In [20]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def get_sentiment(input_text: str, model=model, tokenizer=tokenizer):

    inputs = tokenizer(input_text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    return torch.nn.Softmax(dim=1)(logits)[0].tolist()

def get_stock_data(ticker: str) -> "tuple[torch.tensor(), float, str]|int": # type: ignore
    """
    Retrieves stock data, performs sentiment analysis on related news stories,
    and returns the processed data as a tuple.
    Args:
        ticker (str): The stock ticker symbol.
    Returns:
        tuple: A tuple containing the processed stock data tensor and the annual percent change.
    """
    today: datetime = datetime.today()
    yesteryear: str = (today - timedelta(days=345)).strftime('%Y-%m-%d')
    column_names: "list[str]" = ["low","open","volume","high","close","adjclose","Annual Percent Change","positive","negative","neutral"]
    stock_means: "list[float]" = [526.3256709784156, 543.3051050285828, 1550317.649111653, 560.6335547940919, 543.4596727527701, 432.9478474861315, 2.1972159402834595, 0.26918878308244654, 0.27109889272321963, 0.45971232487752245]
    stock_std: "list[float]" = [25100.799536177463, 25956.76629687626, 13374912.758878224, 26792.649346820093, 25956.514595704837, 20572.229772641494, 477.38693948841035, 0.13956166294058042, 0.1719876441524078, 0.16237148015431316]

    mean_series = pd.Series(stock_means, index=column_names)
    std_series = pd.Series(stock_std, index=column_names)

    try:
        stock_news = Sentiment(ticker)
        sentiment_score = stock_news.get_dataframe(days=1)
        stories = sentiment_score['headline'].tolist()[:10]
        sentiments: "list[list[float]]" = []

        for story in stories:
            sentiment = get_sentiment(story)
            sentiments.append(sentiment)
        sentiments_df = pd.DataFrame(sentiments, columns=['positive', 'negative', 'neutral'])
        mean_sentiments: pd.Series = sentiments_df.mean()
        sentiment_return: str = str(pd.Series.idxmax(mean_sentiments))
        stock_sentiment: "list[float]" = mean_sentiments.values.tolist()
    except:
        sentiment_return = "neutral"
        stock_sentiment = [0.3, 0.3, 0.4]
    try:
        ticker_info: pd.DataFrame = si.get_data(ticker, start_date=yesteryear, end_date=today, interval="1mo")
    except:
        return 404
    ticker_info = ticker_info.drop(columns=['ticker'])

    ticker_info = ticker_info.fillna(method='ffill')
    print(ticker_info['close'])
    ticker_info['Annual Percent Change'] = (ticker_info.iloc[-1]['close'] - ticker_info.iloc[0]['close']) / ticker_info.iloc[0]['close'] * 100
    annualized_ticker_info = ticker_info.mean()

    annualized_ticker_info['positive'] = stock_sentiment[0]
    annualized_ticker_info['negative'] = stock_sentiment[1]
    annualized_ticker_info['neutral'] = stock_sentiment[2]
    annualized_ticker_info = annualized_ticker_info[column_names] # type: ignore

    normalalized_ticker_info = (annualized_ticker_info - mean_series) / std_series
    print(normalalized_ticker_info)
    print(annualized_ticker_info['Annual Percent Change'])

    return torch.Tensor(normalalized_ticker_info.values.tolist()), annualized_ticker_info['Annual Percent Change'], sentiment_return


In [21]:
data = get_stock_data("lulu")

c:\Users\William\AppData\Local\Programs\Python\Python38\lib\site-packages\stocksent\get_sentiment_data.py:97: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  news_df['date'] = pd.to_datetime(news_df.date).dt.date


2022-08-01    299.959991
2022-09-01    279.559998
2022-10-01    329.040009
2022-11-01    380.309998
2022-12-01    320.380005
2023-01-01    306.880005
2023-02-01    309.200012
2023-03-01    364.190002
2023-04-01    379.929993
2023-05-01    331.929993
2023-06-01    378.500000
2023-07-01    378.500000
Name: close, dtype: float64
low                     -0.008708
open                    -0.008254
volume                   2.633342
high                    -0.007328
close                   -0.007908
adjclose                -0.004606
Annual Percent Change    0.050245
positive                 0.220771
negative                 0.168042
neutral                 -0.367751
dtype: float64
26.183494726723907


In [198]:
stock_tickers = pd.read_csv("G:/My Drive/UdS/Classes/Data Science/DS-Miniproject/nasdaq_screener_1687708388821.csv")
stock_tickers = stock_tickers[['Symbol', 'Name']].copy()

print(stock_tickers.head())

stock_names_with_tickers = [f'{stock_tickers["Symbol"][i]} - {stock_tickers["Name"][i]}' for i in stock_tickers.index]
stock_ticker_lookup_dict = {stock_names_with_tickers[i]: stock_tickers['Symbol'][i] for i in stock_tickers.index}

                              

  Symbol                                               Name
0      A             Agilent Technologies Inc. Common Stock
1     AA                    Alcoa Corporation Common Stock 
2    AAC  Ares Acquisition Corporation Class A Ordinary ...
3   AACG   ATA Creativity Global American Depositary Shares
4   AACI            Armada Acquisition Corp. I Common Stock


In [205]:
print(stock_names_with_tickers)
with open("stock_names_with_tickers.txt", "w") as f:
    for stock in stock_names_with_tickers:
        f.write(stock + "\n")


['A - Agilent Technologies Inc. Common Stock', 'AA - Alcoa Corporation Common Stock ', 'AAC - Ares Acquisition Corporation Class A Ordinary Shares', 'AACG - ATA Creativity Global American Depositary Shares', 'AACI - Armada Acquisition Corp. I Common Stock', 'AACIW - Armada Acquisition Corp. I Warrant', 'AACT - Ares Acquisition Corporation II Class A Ordinary Shares', 'AADI - Aadi Bioscience Inc. Common Stock', 'AAIC - Arlington Asset Investment Corp Class A (new)', 'AAIC^B - Arlington Asset Investment Corp 7.00% ', 'AAIC^C - Arlington Asset Investment Corp 8.250% Seies C Fixed-to-Floating Rate Cumulative Redeemable Preferred Stock   ', 'AAIN - Arlington Asset Investment Corp 6.000% Senior Notes Due 2026', 'AAL - American Airlines Group Inc. Common Stock', 'AAM^A - Apollo Asset Management Inc. 6.375% Series A Preferred Stock', 'AAM^B - Apollo Asset Management Inc. 6.375% Series B Preferred Stock', 'AAMC - Altisource Asset Management Corp Com', 'AAME - Atlantic American Corporation Commo

In [208]:
with open('stock_names_with_tickers.txt', 'r') as f:
    companies = f.read().splitlines()
print(companies[:10])

['A - Agilent Technologies Inc. Common Stock', 'AA - Alcoa Corporation Common Stock ', 'AAC - Ares Acquisition Corporation Class A Ordinary Shares', 'AACG - ATA Creativity Global American Depositary Shares', 'AACI - Armada Acquisition Corp. I Common Stock', 'AACIW - Armada Acquisition Corp. I Warrant', 'AACT - Ares Acquisition Corporation II Class A Ordinary Shares', 'AADI - Aadi Bioscience Inc. Common Stock', 'AAIC - Arlington Asset Investment Corp Class A (new)', 'AAIC^B - Arlington Asset Investment Corp 7.00% ']


In [209]:
stock = stock_names_with_tickers[205]
print(stock)
print(stock_ticker_lookup_dict[stock])

ticker_lookup_df = pd.DataFrame.from_dict(stock_ticker_lookup_dict, orient="index")
ticker_lookup_df.to_csv("dicker_lookup_df.csv")

AGEN - Agenus Inc. Common Stock
AGEN


In [210]:
companies = pd.read_csv('dicker_lookup_df.csv', index_col=0)
companies = companies.to_dict("split")
companies = zip(companies["index"], companies["data"])
compnay_dict = {k: v[0] for k, v in companies}
print(compnay_dict)

{'A - Agilent Technologies Inc. Common Stock': 'A', 'AA - Alcoa Corporation Common Stock ': 'AA', 'AAC - Ares Acquisition Corporation Class A Ordinary Shares': 'AAC', 'AACG - ATA Creativity Global American Depositary Shares': 'AACG', 'AACI - Armada Acquisition Corp. I Common Stock': 'AACI', 'AACIW - Armada Acquisition Corp. I Warrant': 'AACIW', 'AACT - Ares Acquisition Corporation II Class A Ordinary Shares': 'AACT', 'AADI - Aadi Bioscience Inc. Common Stock': 'AADI', 'AAIC - Arlington Asset Investment Corp Class A (new)': 'AAIC', 'AAIC^B - Arlington Asset Investment Corp 7.00% ': 'AAIC^B', 'AAIC^C - Arlington Asset Investment Corp 8.250% Seies C Fixed-to-Floating Rate Cumulative Redeemable Preferred Stock   ': 'AAIC^C', 'AAIN - Arlington Asset Investment Corp 6.000% Senior Notes Due 2026': 'AAIN', 'AAL - American Airlines Group Inc. Common Stock': 'AAL', 'AAM^A - Apollo Asset Management Inc. 6.375% Series A Preferred Stock': 'AAM^A', 'AAM^B - Apollo Asset Management Inc. 6.375% Series